[![](images/colab-badge.png)](https://colab.research.google.com/github/fzampirolli/si-md2/blob/main/si-md2/notebooks_alunos/cap02/cap02_aluno.ipynb)

# Mineração de Dados e Regras de Associação

## Referências do Capítulo


Tem que citar neste capítulo: @han2008, @tan2009, @witten2005, @goldschmidt2005


\printbibliography[heading=none]